# NSIDC EGI Interface

- https://github.com/creare-com/podpac/issues/237
- https://developer.earthdata.nasa.gov/sdps/programmatic-access-docs#curl
- https://nsidc.org/support/how/how-do-i-programmatically-request-data-services

In [ ]:
import requests
import xml.etree.ElementTree
import socket
import os

## Obtain Token

https://wiki.earthdata.nasa.gov/display/CMR/Creating+a+Token+Common

In [ ]:
username = ""
password = ""
url = 'https://cmr.earthdata.nasa.gov/legacy-services/rest/tokens'

In [ ]:
def get_ip():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    try:
        s.connect(('8.8.8.8', 80))
        ip = s.getsockname()[0]
    except:
        ip = '127.0.0.1'
    finally:
        s.close()
    return ip

In [ ]:
ip = get_ip()
request = """
<token>
    <username>{username}</username>
    <password>{password}</password>
    <client_id>podpac</client_id>
    <user_ip_address>{ip}</user_ip_address>
</token>
""".format(username=username, password=password,ip=ip)
headers = {"Content-Type": "application/xml"}
r = requests.post(url, data=request, headers=headers)

In [ ]:
tree = xml.etree.ElementTree.fromstring(r.text)
token = [element.text for element in tree.findall('id')][0]

In [ ]:
token

## Get Data

In [ ]:
base_url = "https://n5eil01u.ecs.nsidc.org/egi/request"
# base_url = "https://n5eil02u.ecs.nsidc.org/egi/request"
product = "SPL3SMA"
version = "003"
start_time = "2015-03-30"
end_time = "2015-06-12"
data_layers = "/Soil_Moisture_Retrieval_Data/soil_moisture"
bbox = "-70,42,-69,43"
response_format = "HDF-EOS5"
# response_format = "GeoTIFF"
token = token  # from above

In [ ]:
url = "{base_url}?short_name={product}&version={version}".format(base_url=base_url, product=product, version=version,) + \
      "&time={start_time},{end_time}&Coverage={data_layers}".format(start_time=start_time, end_time=end_time, data_layers=data_layers, ) + \
      "&Bbox={bbox}&format={response_format}&token={token}".format(bbox=bbox, response_format=response_format, token=token)

r = requests.get(url)

## Look at the Data

In [ ]:
from podpac.data import Rasterio
import zipfile
from io import BytesIO
%matplotlib inline

In [ ]:
with open('output.zip', 'wb') as f:
    f.write(r.content)

In [ ]:
f = BytesIO(r.content)
zip_file = zipfile.ZipFile(f)

for name in zip_file.namelist():
    h5data = zip_file.read(name)


In [ ]:
with zipfile.ZipFile("output.zip","r") as zip_ref:
    zip_ref.extractall()

In [ ]:
node = Rasterio(source='78753123/SMAP_L3_SM_A_20150413_R13080_002_Soil_Moisture_Retrieval_Data_soil_moisture_1aec95e7.tif')

In [ ]:
node.native_coordinates

In [ ]:
node.native_coordinates.crs

In [ ]:
o = node.eval(node.native_coordinates)
o.plot()

## PODPAC Interface

In [ ]:
from podpac.datalib import EGI
from podpac import settings

In [ ]:
username = ""
password = ""
token = "DCF3DD10-C56C-1F7F-12FE-6408839955B8"

In [ ]:
node = EGI(token=token, short_name="SPL3SMA", version="003", data_key="/Soil_Moisture_Retrieval_Data/soil_moisture")

In [ ]:
node

In [ ]:
node.token_valid()

In [ ]:
node.get_token()

In [ ]:
node.token_valid()

In [ ]:
node.token